In [1]:
import pandas as pd
import numpy as np
import json

In [90]:
# !head data/mfp-diaries.tsv

## Exploratory Data Analysis

In [87]:
df = pd.read_csv('data/mfp-diaries.tsv',
                  sep='\t',
                  header=None,
                  names=['userId','diary_date','food_entries','daily_goal'])

In [88]:
df.head()

,userId,diary_date,food_entries,daily_goal
0,1,2014-09-14,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [89]:
len(df['userId'].unique())

9896

In [118]:
entry = json.loads(df.iloc[2, 2])
entry

[{'meal': 'MY food',
  'dishes': [{'nutritions': [{'name': 'Calories', 'value': '147'},
     {'name': 'Carbs', 'value': '36'},
     {'name': 'Fat', 'value': '0'},
     {'name': 'Protein', 'value': '0'},
     {'name': 'Sodium', 'value': '306'},
     {'name': 'Sugar', 'value': '36'}],
    'name': 'Gatorade - Lemon-Lime (Australia), 600 ml'},
   {'nutritions': [{'name': 'Calories', 'value': '412'},
     {'name': 'Carbs', 'value': '29'},
     {'name': 'Fat', 'value': '24'},
     {'name': 'Protein', 'value': '21'},
     {'name': 'Sodium', 'value': '258'},
     {'name': 'Sugar', 'value': '29'}],
    'name': 'my - McDonalds Espresso Pronto® Flat White, 2 TALL'},
   {'nutritions': [{'name': 'Calories', 'value': '176'},
     {'name': 'Carbs', 'value': '33'},
     {'name': 'Fat', 'value': '1'},
     {'name': 'Protein', 'value': '5'},
     {'name': 'Sodium', 'value': '195'},
     {'name': 'Sugar', 'value': '0'}],
    'name': 'Uncle Tobys Australia - Vita Brits, 3 Biscuits 33.3g'},
   {'nutritions

In [119]:
entry[0].keys()

dict_keys(['meal', 'dishes', 'sequence'])

In [120]:
entry[0]['dishes'][1]

{'nutritions': [{'name': 'Calories', 'value': '412'},
  {'name': 'Carbs', 'value': '29'},
  {'name': 'Fat', 'value': '24'},
  {'name': 'Protein', 'value': '21'},
  {'name': 'Sodium', 'value': '258'},
  {'name': 'Sugar', 'value': '29'}],
 'name': 'my - McDonalds Espresso Pronto® Flat White, 2 TALL'}

## UserId, Entry_Date, Daily_Goal  (Need to Parse Data)

In [14]:
goals = json.loads(df.iloc[0, 3])
goals

{'total': [{'name': 'Calories', 'value': 2924},
  {'name': 'Carbs', 'value': 340},
  {'name': 'Fat', 'value': 114},
  {'name': 'Protein', 'value': 186},
  {'name': 'Sodium', 'value': 3658},
  {'name': 'Sugar', 'value': 109}],
 'goal': [{'name': 'Calories', 'value': 3173},
  {'name': 'Carbs', 'value': 396},
  {'name': 'Fat', 'value': 105},
  {'name': 'Protein', 'value': 160},
  {'name': 'Sodium', 'value': 2300},
  {'name': 'Sugar', 'value': 119}]}

In [15]:
df_goal = df.drop(['food_entries'], axis=1)

In [16]:
df_goal.head()

,userId,entry_date,daily_goal
0,1,2014-09-14,"{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [85]:
parse_data = df_goal.drop(['userId', 'entry_date'], axis=1)
parse_data.tail()

,daily_goal
587182,"{""total"": [{""name"": ""Calories"", ""value"": 1979}..."
587183,"{""total"": [{""name"": ""Calories"", ""value"": 2141}..."
587184,"{""total"": [{""name"": ""Calories"", ""value"": 543},..."
587185,"{""total"": [{""name"": ""Calories"", ""value"": 2024}..."
587186,"{""total"": [{""name"": ""Calories"", ""value"": 1496}..."


In [86]:
ex_dict = parse_data['daily_goal'][0]
type(ex_dict)

str

In [78]:
ex_dict

'{"total": [{"name": "Calories", "value": 2924}, {"name": "Carbs", "value": 340}, {"name": "Fat", "value": 114}, {"name": "Protein", "value": 186}, {"name": "Sodium", "value": 3658}, {"name": "Sugar", "value": 109}], "goal": [{"name": "Calories", "value": 3173}, {"name": "Carbs", "value": 396}, {"name": "Fat", "value": 105}, {"name": "Protein", "value": 160}, {"name": "Sodium", "value": 2300}, {"name": "Sugar", "value": 119}]}'

In [21]:
json.loads(ex_dict)

{'total': [{'name': 'Calories', 'value': 2924},
  {'name': 'Carbs', 'value': 340},
  {'name': 'Fat', 'value': 114},
  {'name': 'Protein', 'value': 186},
  {'name': 'Sodium', 'value': 3658},
  {'name': 'Sugar', 'value': 109}],
 'goal': [{'name': 'Calories', 'value': 3173},
  {'name': 'Carbs', 'value': 396},
  {'name': 'Fat', 'value': 105},
  {'name': 'Protein', 'value': 160},
  {'name': 'Sodium', 'value': 2300},
  {'name': 'Sugar', 'value': 119}]}

### Goal:
Create 1 column per dictionary in both total key and goal key

- total
    + total_calories
    + total_carbs
    + total_fat
    + total_protein
    + total_sodium
    + total_sugar
- goal
    + goal_calories
    + goal_carbs
    + goal_fat
    + goal_protein
    + goal_sodium
    + goal_sugar

In [23]:
json_dict = json.loads(ex_dict)
json_dict

{'total': [{'name': 'Calories', 'value': 2924},
  {'name': 'Carbs', 'value': 340},
  {'name': 'Fat', 'value': 114},
  {'name': 'Protein', 'value': 186},
  {'name': 'Sodium', 'value': 3658},
  {'name': 'Sugar', 'value': 109}],
 'goal': [{'name': 'Calories', 'value': 3173},
  {'name': 'Carbs', 'value': 396},
  {'name': 'Fat', 'value': 105},
  {'name': 'Protein', 'value': 160},
  {'name': 'Sodium', 'value': 2300},
  {'name': 'Sugar', 'value': 119}]}

In [24]:
json_dict.keys()

dict_keys(['total', 'goal'])

In [26]:
json_dict['total']

[{'name': 'Calories', 'value': 2924},
 {'name': 'Carbs', 'value': 340},
 {'name': 'Fat', 'value': 114},
 {'name': 'Protein', 'value': 186},
 {'name': 'Sodium', 'value': 3658},
 {'name': 'Sugar', 'value': 109}]

In [27]:
json_dict['total'][0]

{'name': 'Calories', 'value': 2924}

In [127]:
# json_dict.keys()[0]
# Gives Error: 'dict_keys' object does not support indexing, hence needs to be set to a list to iterate.

In [125]:
total_cols = [list(json_dict.keys())[0] + "_" + i['name'].lower() 
              for i in json_dict['total']]
total_cols

['total_calories',
 'total_carbs',
 'total_fat',
 'total_protein',
 'total_sodium',
 'total_sugar']

In [53]:
total_values = [x['value'] for x in json_dict['total']]
total_values

[2924, 340, 114, 186, 3658, 109]

In [54]:
goal_cols = [list(json_dict.keys())[1] + "_" + k['name'].lower()
                for glove in json_dict['goal']]
goal_cols

['goal_calories',
 'goal_carbs',
 'goal_fat',
 'goal_protein',
 'goal_sodium',
 'goal_sugar']

In [55]:
goal_values = [x['value'] for x in json_dict['goal']]
goal_values

[3173, 396, 105, 160, 2300, 119]

### Combined Keys and Values for *Total* and *Goals* into single dictionary

In [56]:
total_dict = {}
for col, val in zip(total_cols, total_values):
    total_dict[col] = val
total_dict

{'total_calories': 2924,
 'total_carbs': 340,
 'total_fat': 114,
 'total_protein': 186,
 'total_sodium': 3658,
 'total_sugar': 109}

In [57]:
goal_dict = {}
for col, val in zip(goal_cols, goal_values):
    goal_dict[col] = val
goal_dict

{'goal_calories': 3173,
 'goal_carbs': 396,
 'goal_fat': 105,
 'goal_protein': 160,
 'goal_sodium': 2300,
 'goal_sugar': 119}

In [138]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res

In [141]:
combined_dict = Merge(total_dict, goal_dict)
combined_dict

{'total_calories': 2924,
 'total_carbs': 340,
 'total_fat': 114,
 'total_protein': 186,
 'total_sodium': 3658,
 'total_sugar': 109,
 'goal_calories': 3173,
 'goal_carbs': 396,
 'goal_fat': 105,
 'goal_protein': 160,
 'goal_sodium': 2300,
 'goal_sugar': 119}

In [140]:
dict3 = pd.DataFrame.from_dict(dict3, orient='index').T
dict3

,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar
0,2924,340,114,186,3658,109,3173,396,105,160,2300,119


### Alternative way

In [58]:
total = pd.DataFrame.from_dict(total_dict, orient='index').T

In [59]:
goal = pd.DataFrame.from_dict(goal_dict, orient='index').T

In [73]:
total_goal_concat = pd.concat([total, goal], sort=False, axis=1)
total_goal_concat

,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar
0,2924,340,114,186,3658,109,3173,396,105,160,2300,119


## Using Function to loop through entire dataset

In [80]:
df_goal.head()

,userId,entry_date,daily_goal
0,1,2014-09-14,"{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [135]:
df_goal = df_goal.drop(['daily_goal'], axis=1)

In [130]:
df_goal.head()

,userId,entry_date
0,1,2014-09-14
1,1,2014-09-15
2,1,2014-09-16
3,1,2014-09-17
4,1,2014-09-18


In [132]:
full_df = pd.concat([df_goal, total_goal_concat], axis=1, sort=False)
full_df.head().T

,0,1,2,3,4
userId,1,1,1,1,1
entry_date,2014-09-14,2014-09-15,2014-09-16,2014-09-17,2014-09-18
total_calories,2924,NaN,NaN,NaN,NaN
total_carbs,340,NaN,NaN,NaN,NaN
total_fat,114,NaN,NaN,NaN,NaN
total_protein,186,NaN,NaN,NaN,NaN
total_sodium,3658,NaN,NaN,NaN,NaN
total_sugar,109,NaN,NaN,NaN,NaN
goal_calories,3173,NaN,NaN,NaN,NaN
goal_carbs,396,NaN,NaN,NaN,NaN


## Transforming dictionary to dataframe

In [32]:
goal = {'total_calories': 123, 'goal_calories': 456}
goal

{'total_calories': 123, 'goal_calories': 456}

In [33]:
goal_df = pd.DataFrame.from_dict({'total_calories': 5, 'total_carbs': 2,'total_fat': 1,
                                  'total_protein': 23,'total_sodium': 12, 'total_sugar':30,
                                  'goal_calories': 5, 'goal_carbs': 1,'goal_fat': 0,
                                  'goal_protein': 20,'goal_sodium': 7, 'goal_sugar': 15}, orient='index').T
goal_df

,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar
0,5,2,1,23,12,30,5,1,0,20,7,15


In [34]:
total_list = list(goal_df)
total_list.remove('goal_calories', 'goal_carbs')
total_list

TypeError: remove() takes exactly one argument (2 given)